# Hackathon

Импортируем модули

In [295]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import catboost as ctb
import tensorflow as tf

Читаем файлы для трэйна и теста

In [264]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Для начала разберемся какие типы данных у нас присутствуют 

In [265]:
df_train.dtypes

Year                   int64
Date                  object
Locality              object
Address               object
Estimated Value      float64
Sale Price           float64
Property              object
Residential           object
num_rooms              int64
carpet_area            int64
property_tax_rate    float64
dtype: object

In [266]:
df_train

,Year,Date,Locality,Address,Estimated Value,Sale Price,Property,Residential,num_rooms,carpet_area,property_tax_rate
0,2009,2009-01-02,Greenwich,40 ETTL LN UT 24,711270.0,975000.0,Condo,Condominium,2,760,1.025953
1,2009,2009-01-02,East Hampton,18 BAUER RD,119970.0,189900.0,Single Family,Detached House,3,921,1.025953
2,2009,2009-01-02,Ridgefield,48 HIGH VALLEY RD.,494530.0,825000.0,Single Family,Detached House,3,982,1.025953
3,2009,2009-01-02,Old Lyme,56 MERIDEN RD,197600.0,450000.0,Single Family,Detached House,3,976,1.025953
4,2009,2009-01-02,Naugatuck,13 CELENTANO DR,105440.0,200000.0,Single Family,Detached House,3,947,1.025953
...,...,...,...,...,...,...,...,...,...,...,...
553947,2022,2022-12-31,Enfield,50 SWORD AVE,122420.0,198000.0,Residential,Detached House,3,1009,1.003979
553948,2022,2022-12-31,Naugatuck,10 ROLLINGWOOD DR,139180.0,319900.0,Residential,Detached House,3,999,1.003979
553949,2022,2022-12-31,South Windsor,189 PALMER DR,200000.0,305000.0,Residential,Detached House,3,1017,1.003979
553950,2022,2022-12-31,Bloomfield,33 BURNWOOD DRIVE,195440.0,271000.0,Residential,Detached House,3,1061,1.003979


У нас присутствуют категориальные признаки(Locality, Property, Residential), также есть дата, и множество числовых данных. Для начала разберемся с датой, год уже присутствует, остается только вытащить месяц и день, это потенциально может повлиять на цену, ну и затем уберем дату и адрес(они нам больше не понадобятся)

In [267]:
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['month']= df_train['Date'].dt.month
df_train['day']= df_train['Date'].dt.day
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['month']= df_test['Date'].dt.month
df_test['day']= df_test['Date'].dt.day

In [269]:
df_train.drop(['Date','Address'], axis=1, inplace=True)
df_test.drop(['Date','Address'], axis=1, inplace=True)

Дальше разберемся с категориальными признаками. Property и Residential ввиду малого кол-ва уникальных значений просто воспользуемся OneHotEncoder, а с Locality так не прокатит(там 170 уникальных значений), к нему применим обычный LabelEncoder

In [272]:
df_train[['Locality']].nunique(), df_train[['Property']].nunique(), df_train[['Residential']].nunique()

(Locality    170
 dtype: int64,
 Property    6
 dtype: int64,
 Residential    5
 dtype: int64)

In [275]:
# ohe = OneHotEncoder()
# transformed = ohe.fit_transform(df_ohe[['Locality']]) 
ohe1 = OneHotEncoder()
transformed1 = ohe1.fit_transform(df_train[['Property']]) 
ohe2 = OneHotEncoder()
transformed2 = ohe2.fit_transform(df_train[['Residential']]) 

In [276]:
# df_ohe[ohe.categories_[0]] = transformed.toarray()
df_train[ohe1.categories_[0]] = transformed1.toarray()
df_train[ohe2.categories_[0]] = transformed2.toarray()

In [277]:
lb = LabelEncoder()
df_train['Locality'] = lb.fit_transform(df_train['Locality'])

In [278]:
df_train.drop(['Property'], axis=1, inplace=True)

Осталось только разобраться с числовыми признаками, необходимо стандартизировать Sale price и Estimated Value, это улучшит работу модели и нам никак не помешает

In [279]:
scaler = StandardScaler()
df_train["Estimated Value"] = scaler.fit_transform(df_ohe_w["Estimated Value"].values.reshape(-1,1))
df_train["Sale Price"] = scaler.fit_transform(df_ohe_w["Sale Price"].values.reshape(-1,1))

Разобьем данные на train и validation выборки

In [280]:
x_train, x_test = train_test_split(df_ohe_w.drop('Sale Price', axis=1), train_size=0.7, random_state=1)
y_train, y_test = train_test_split(df_ohe_w['Sale Price'], train_size=0.7, random_state=1)

Я собираюсь построить множество различных моделей(Catboost, RandomForest, Linear) и нейронку для регрессии

In [282]:
model = RandomForestRegressor(n_jobs=-1, verbose=10, random_state=27)
# Для начала обучим модель без настройки гиперпараметров
model.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.4s


building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.2s


...

building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100


[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   52.0s


building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed:  1.0min remaining:    8.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished


RandomForestRegressor(n_jobs=-1, verbose=10)

In [287]:
pred_test = model.predict(x_test)
pred_train = model.predict(x_train)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    1.6s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    3.3s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    5.3s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    7.5s
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed:   10.8s remaining:    1.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:   12.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    1.4s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    3.0s
[P

In [288]:
r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train)

(0.9601139397338165, 0.0433383560636188)

In [289]:
r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test)

(0.5041485547993616, 0.11620408223179166)

Всего 50% дисперсии на тесте и 96% на трейне, признаки переобучения, попробуем настроить max_depth

In [ ]:
param = [10, 15, 20, 30, 40]
res=[]
for i in param:
    model = RandomForestRegressor(n_jobs=-1, verbose=10, random_state=27)
    
    model.fit(x_train, y_train)
    
    pred_test = model.predict(x_test)
    pred_train = model.predict(x_train)
    
    res.append((r2_score(y_train, pred_train), r2_score(y_test, pred_test)))

In [308]:
res

[(0.8370303216446241, 0.5414108541853),
 (0.9463678167223394, 0.5167737982438512),
 (0.9535759464051375, 0.5190224180178811),
 (0.9551932203133116, 0.5100959972256005),
 (0.9555948645889194, 0.5147110741669813)]

Лучше всего себя показал результат с max_depth 10, дальше настроим min_samples_leaf

In [297]:
tree = RandomForestRegressor(max_depth=15, n_jobs=-1, random_state=27, verbose=10)

param_grid = {'min_samples_leaf': [1, 5, 10, 15, 20]}
cv = 5

grid_search = GridSearchCV(tree, param_grid=param_grid, cv=cv)
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed:   43.9s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   45.9s finished


Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.6s
[Parall

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.3s
[Parall

...

[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   31.8s


building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed:   36.5s remaining:    4.9s


{'min_samples_leaf': 1}


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   39.3s finished


Лучше всего оставить min_samples_leaf без изменения

In [304]:
model = RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=27, verbose=10)

model.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.4s


building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.6s


...

building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100


[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   31.6s


building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed:   36.4s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   38.9s finished


RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=27, verbose=10)

In [305]:
pred_test = model.predict(x_test)
pred_train = model.predict(x_train)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[P

In [302]:
r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train)

(0.8370303216446241, 0.11481737226641192)

In [303]:
r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test)

(0.5414108541853, 0.12016241102955724)

С настроенными гиперпараметрами результат улучшился на 4 процента(

Пусть это будет первая модель для отправки

In [311]:
df_test['Locality'] = lb.transform(df_test['Locality'])
transformed_t = ohe1.transform(df_test[['Property']]) 
transformed_t1 = ohe2.transform(df_test[['Residential']]) 
df_test[ohe1.categories_[0]] = transformed_t.toarray()
df_test[ohe2.categories_[0]] = transformed_t1.toarray()
df_test.drop('Property', axis=1, inplace=True)

In [313]:
df_test['Estimated Value'] = scaler.transform(df_test["Estimated Value"].values.reshape(-1,1))

In [314]:
df_test_for_model = df_test.drop(['Segment', 'Sale Price'], axis=1)

In [316]:
pred = model.predict(df_test_for_model)

df_test['Sale Price'] = pred

df_test['Profit'] = (df_test['Sale Price'] - df_test['Estimated Value'])/100

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


Осталось только разбить данные на сегменты, чтобы стандартизация нам никак не помешала, разобьем данные с помощью квантилей

In [319]:
q1, q2, q3 = df_test['Profit'].quantile([0.25, 0.5, 0.75])
df_test['Segment'] = np.where(df_test['Profit'] >= q3, 0, np.where(df_test['Profit'] >= q2, 1, np.where(df_test['Profit'] >= q1, 2, 3)))

In [320]:
df_sub = pd.read_csv('submission.csv')

df_sub['Segment'] = df_test['Segment'] 

df_sub.to_csv('my_submission_file.csv', index=False)

Данный способ позволяет предсказать около 26% сегментов, что на самом деле очень плохо попробуем Catboost model 

В catboost изначально стоят неплохо настроенные гиперпараметры, так что сразу обучаем

In [321]:
model = ctb.CatBoostRegressor(random_state=27, verbose=1)

model.fit(x_train, y_train)

Learning rate set to 0.104996
0:	learn: 0.9255216	total: 40.7ms	remaining: 40.7s
1:	learn: 0.8855645	total: 65.1ms	remaining: 32.5s
2:	learn: 0.8508343	total: 89.7ms	remaining: 29.8s
3:	learn: 0.8216767	total: 114ms	remaining: 28.5s
4:	learn: 0.7972405	total: 138ms	remaining: 27.4s
5:	learn: 0.7768142	total: 164ms	remaining: 27.1s
6:	learn: 0.7597265	total: 189ms	remaining: 26.8s
7:	learn: 0.7456482	total: 217ms	remaining: 27s
8:	learn: 0.7341220	total: 243ms	remaining: 26.8s
9:	learn: 0.7242128	total: 267ms	remaining: 26.4s
10:	learn: 0.7156154	total: 291ms	remaining: 26.2s
11:	learn: 0.7089221	total: 315ms	remaining: 26s
12:	learn: 0.7027079	total: 339ms	remaining: 25.7s
13:	learn: 0.6976428	total: 363ms	remaining: 25.6s
14:	learn: 0.6932453	total: 390ms	remaining: 25.6s
15:	learn: 0.6897707	total: 415ms	remaining: 25.5s
16:	learn: 0.6867302	total: 444ms	remaining: 25.7s
17:	learn: 0.6840303	total: 470ms	remaining: 25.6s
18:	learn: 0.6814188	total: 496ms	remaining: 25.6s
19:	learn: 0

In [322]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)

In [323]:
print(r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train))

0.8359267477858133 0.11364517014080265


In [324]:
print(r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test))

0.517034593123481 0.1210294094835803


51% дисперсии при обучении, но если отправить при остальных тех же условиях accuracy достигает 30%

Линейная модель умудряется описать только 10% дисперсии(это очень мало), так что перейду сразу к нейронкам

Долго настраивая слои, активации и rate в дропоуте у меня получилась такая нейронка

In [329]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=x_train.iloc[0].shape, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.01),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(rate=0.05),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.05),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


In [330]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=[tf.keras.metrics.mean_squared_error,
                       tf.keras.metrics.mean_absolute_percentage_error
                       ])

In [331]:
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


x_train, x_val, x_test = train_val_test_split(df_ohe_w.drop('Sale Price', axis=1))
y_train, y_val, y_test = train_val_test_split(df_ohe_w['Sale Price'])

In [332]:
annealing = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.1,
                                                 patience=5,
                                                 verbose=1,
                                                 min_delta=0.001)

In [333]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/first', 
                                             histogram_freq=1)

In [262]:
model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          batch_size = 1000,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=20), annealing, tb_callback],
          epochs=100)

Epoch 1/100
388/388 [==============================] - 5s 14ms/step - loss: 0.1693 - mean_squared_error: 0.4839 - mean_absolute_percentage_error: 197.1369 - val_loss: 0.1395 - val_mean_squared_error: 1.7502 - val_mean_absolute_percentage_error: 162.4903 - lr: 1.0000e-14
Epoch 2/100
388/388 [==============================] - 5s 13ms/step - loss: 0.1685 - mean_squared_error: 0.4915 - mean_absolute_percentage_error: 188.9921 - val_loss: 0.1393 - val_mean_squared_error: 2.1507 - val_mean_absolute_percentage_error: 164.3531 - lr: 1.0000e-14
Epoch 3/100
388/388 [==============================] - 5s 14ms/step - loss: 0.1684 - mean_squared_error: 0.4849 - mean_absolute_percentage_error: 193.8125 - val_loss: 0.1397 - val_mean_squared_error: 1.8431 - val_mean_absolute_percentage_error: 164.9094 - lr: 1.0000e-14
Epoch 4/100
388/388 [==============================] - 6s 15ms/step - loss: 0.1682 - mean_squared_error: 0.4877 - mean_absolute_percentage_error: 193.1836 - val_loss: 0.1402 - val_mean_sq

In [259]:
pred = model.predict(x_test)

2597/2597 [==============================] - 4s 2ms/step


In [260]:
r2_score(y_test, pred)

0.31731232675449605

Нейронка с трудом описывает 30% дисперсии(не подходит), на последок решил себя проверить использовав встроенную нейронку из sklearn, она описала лишь 27% дисперсии(все мимо)

In [62]:
from sklearn.neural_network import MLPRegressor

In [ ]:
model = MLPRegressor(learning_rate='adaptive', random_state=27, max_iter=500, verbose=1, n_iter_no_change=50)

model.fit(x_train, y_train)

In [65]:
pred = model.predict(x_test)

In [66]:
r2_score(y_test, pred)

0.2756038968572416

## Results

In [337]:
pd.DataFrame([[0.09673426871284, '<5%'],[0.31731232675449605,'15%'], [0.2756038968572416, '15%'],[0.5414108541853, '26%-27%'],[0.517034593123481, '30%']], columns=['R2_score', 'Accuracy_on_segm'], index=['Linear', 'DeepLearn', 'MLPReg', 'RandomForest', 'CatBoost'])

,R2_score,Accuracy_on_segm
Linear,0.096734,<5%
DeepLearn,0.317312,15%
MLPReg,0.275604,15%
RandomForest,0.541411,26%-27%
CatBoost,0.517035,30%
